# Introduction:
<br>
This is my final project on Supervised Learning. My Model building was okay, unfortunately I got mistaken the balance status of my data hence my evaluation metric was in the wrong direction. Next step, I will take time to improve this after end of my course in end December.

## ML Final Project

**Scenario:** You work at a multinational bank that is aiming to increase it's market share in 
Europe. Recently, it has been noticed that the number of customers using the banking 
services has declined, and the bank is worried that existing customers have stopped 
using them as their main bank. <br> 

As a data scientist, you are tasked with finding out the 
reasons behind customer churn (when a customer stops using them as the main bank) and to predict customer churn. <br> 

The marketing team, 
in particular, is interested in your findings and want to better understand existing 
customer behavior and possibly predict customer churn. Your results will help the 
marketing team to use their budget wisely to target potential churners. To achieve 
this objective, in this exercise, you will import the banking data (Churn_Modelling.csv) 
provided by the bank and do some machine learning to solve their problem.

Data dictionary

- CustomerID: Unique ID of each customer
- CredRate: Credit Score of the customer 
- Geography: Country customer is from 
- Gender
- Age
- Tenure: How long customer has been with bank 
- balance: The amount of money customer has/had with bank
- Prod Number: Number of products customer has with bank 
- HasCrCard: Does customer have credit card
- ActMem: Is customer active member / whether customer was actively engaged with bank activities.
- Estimated salary: Annual estimated salary of customer 
- Exited: Whether customer has churned (1 is yes)

# 1) Introduction

The aim for this study outcome is to explore and identify what could be the potential factors that influence/impact behind customer churn. Then build a prediction model to help predict and classify the potential customer that may be churn. Based on the performance, analysis and provide insights to the business on what kind of actions can be taken to minimize the chances of customer churn in order to help increase it's market share in Europe going forward.

# 2) Data understanding/ Preprocessing

In [ ]:
#import libraries

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn import metrics
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error, roc_auc_score, roc_curve
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data = pd.read_csv("../input/customer-churn/Churn_Modelling.csv")

## Understanding Data

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data[data.isna().any(axis=1)]

In [ ]:
data.describe()

In [ ]:
data.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.heatmap(data.corr(),cmap='coolwarm', annot=True, ax=ax)

From the above heatmap, we could get a sense the correlation of the data features. Surprisingly these features their correlation aren't that high. Only "Age" seems has a higher number of correlation of "Exited" feature.<br>
<br>

In [ ]:
#Checking how many countries inclusive
data.Geography.unique()

In [ ]:
#Checking the numberic of Tenure
data.Tenure.unique()

In [ ]:
#Checking the numberic of Prod Number
data["Prod Number"].unique()

In [ ]:
#Checking the indication for Has and do not has Credit Card, just 1 & 0
data.HasCrCard.unique()

In [ ]:
#Don't see how CustomerId is helpful on the ML model, decided to drop it
data = data.drop(["CustomerId"], axis = 1)
data.head()

## Handling Missing Value and check their number of values

Since the number of missung Value is that small amount, decided to drop all the rows with Missing Value.

In [ ]:
data = data.dropna()

In [ ]:
data.Gender.isna().sum()

In [ ]:
data['Gender'].value_counts()

In [ ]:
data['Geography'].value_counts()

In [ ]:
data['Exited'].value_counts()

In [ ]:
data['HasCrCard'].value_counts()

In [ ]:
data['Prod Number'].value_counts()

In [ ]:
data['ActMem'].value_counts()

In [ ]:
#Inspect again if any na value

data.isna().sum()

In [ ]:
data.shape

## Exploratory Data Analysis

In [ ]:
sns.pairplot(data)

In [ ]:
labels = 'Exited', 'Not Exited'
sizes = data['Exited'].value_counts()
explode = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, explode = explode,autopct='%1.1f%%',
        shadow=True, startangle=60)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
import plotly.graph_objs as go

value = data['Exited'].value_counts()

fig = go.Figure(data=[go.Pie(labels=['Exited','Not Exited'],
                            values=data['Exited'].value_counts(),
                            textinfo = 'value + percent + label')])
fig.show()

From the Pie Charts above, it gave us a quick overview of the ratio of customer churn and indeed the percentage of "Exited" group is high and almost close to 80% of the total number.

In [ ]:
# to get offline plotly/HTML
import plotly.offline as offline
offline.init_notebook_mode(connected=True)
 
offline.plot(fig)

In [ ]:
sns.countplot(x = 'Exited', hue = 'Gender', data = data)

Tried to explore what kind of plot will able to give me the overview of each of the features relation to the churn. I found countplot is the best to do comparison so I take it to the next step and compare a few more features which created the plots below.

In [ ]:
fig, axes = plt.subplots(2,3, figsize = (15,10))

sns.countplot(x='Geography', hue = 'Exited',data = data, ax = axes[0,0])
sns.countplot(x='Gender', hue = 'Exited',data = data, ax = axes[0,1])
sns.countplot(x='Tenure', hue = 'Exited',data = data, ax = axes[0,2])
sns.countplot(x='Prod Number', hue = 'Exited',data = data, ax = axes[1,0])
sns.countplot(x='HasCrCard', hue = 'Exited',data = data, ax = axes[1,1])
sns.countplot(x='ActMem', hue = 'Exited',data = data, ax = axes[1,2])

From these 6 Features countplots, here are my hypothesis on their correlation that in relates to the customer churn. <br>

- <b>Geography</b>
    - We can see that the ratio for Germany is smaller ratio of the differences compare to France
- <b>Gender</b>
    - Male has the higher not exited rate and lower exited rate compared to Female
- <b>Tenure</b>
    - The comparison on this feature rather balances, this could suggest that tenure may not be the factors of customer churn. Will find out more along the process.
- <b>Prod Number</b>
    - At a glance, it looks like while the Prod Number 1 & 2 of not exited are way higher, at the same time these numbers may also not be necessary the cause of customer churn.
- <b>HasCrCard</b>
    - Despite "has credit card members" has higher numbers of not exited customer but at the same time the exited customer also pretty high.
- <b>ActMem</b>
    - Surprising the non-Active Member exited is much higher than the Active Member, it may be worth a in depth study by the business.

As Countplot hard to present the correlation of Age and the likelihood of customer churn so decided to use the regplot to study the data.

In [ ]:
AgePlt = sns.regplot(x="Age", y = "Exited", data = data, logistic=True)

# plotting the horiozontal & Vertical line to give a sense on if at threshold of 0.5, customer with age above 62 are likelihood to churn.
AgePlt.axhline(0.5, ls='--', linewidth=1, color='red')
AgePlt.axvline(62, ls='--', linewidth=1, color='red')


As we can see from the plot, the higher of the age the likelihood to churn/exit from the bank. The prediction models should able to find out if this feature is the biggest factor for customer churn.

### Data preprocessing: Encoding

Breaking down the factors to understand what kinds of data are they:
- CredRate: Credit Score of the customer <b> - Integer</b> 
- Geography: Country customer is from <b>- Category</b>
  - 1 = France
  - 2 = Germany
  - 2 = Spain
- Gender  <b>- Category</b>
  - 1 = male
  - 2 = female
- Age<b> - Integer</b> 
- Tenure: How long customer has been with bank <b> - Integer</b> 
- balance: The amount of money customer has/had with bank<b> - Integer</b> 
- Prod Number: Number of products customer has with bank <b> - Integer/Category</b> 
- HasCrCard: Does customer have credit card <b>- Category</b>
  - 0 = No
  - 1 = Yes
- ActMem: Is customer active member / whether customer was actively engaged with bank activities. <b>- Category</b>
  - 0 = No
  - 1 = Yes
- Estimated salary: Annual estimated salary of customer <b> - Integer</b> 
- Exited: Whether customer has churned (1 is yes) <b>- Category</b>
  - 0 = Not Exited
  - 1 = Exited

In [ ]:
data.head()

In [ ]:
final_data = pd.get_dummies(data, columns=["Geography",
                                           "Gender",
                                           "Prod Number",
                                          "HasCrCard",
                                          "ActMem"])
final_data.head()

In [ ]:
final_data.rename({"Geography_France": "France", 
                   "Geography_Germany": "Germany",
                   "Geography_Spain": "Spain",
                   "Gender_Female":"Female",
                   "Gender_Male":"Male",
                   "Prod Number_1":"ProdNum01",
                   "Prod Number_2":"ProdNum02",
                   "Prod Number_3":"ProdNum03",
                   "Prod Number_4":"ProdNum04",
                   "HasCrCard_0":"NoCrCard",
                   "HasCrCard_1":"HasCrCard",
                   "ActMem_0": "NotActMem",
                   "ActMem_1": "ActMem",
                  }, axis='columns', inplace=True)

In [ ]:
final_data.head()

Based on the kinds of data, decided to encode all the categorical factors but since "HasCrCard","ActMem" and"Exited" columns already in binary number so I leave it as it is. <br>
<br>
"Prod Number" is kind of integar & categorical in my opinion, nevertheless decided to encode it to increase the accuracy.


<br>

## Scaling/Feature Engineering

In [ ]:
X = final_data.drop(columns=["Exited"])
y = final_data["Exited"]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Splitting Dataset to train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state = 123)

In [ ]:
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)


In [ ]:
y.shape

In [ ]:
X.shape

# 3 & 4) Machine learning model training & Evaluation/Results

## Logistic Regression

Choosing Logistic Regression simply it is commonly use to predict the target of categorical or binary and in which case our dataset and the outcome trying to achieve suggest that this algorithm is one of the simplest to begin with.

In [ ]:
classifier = LogisticRegression(random_state=123)

classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred

In [ ]:
# Confusion Matrix 
cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot = True)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
LG_acc = accuracy_score(y_pred, y_test)*100
print("Logistic Regression Accuracy: {}".format(LG_acc))

In [ ]:
LG_auc = roc_auc_score(y_test, y_pred)*100
print("Logistic Regression AUC: {}".format(LG_auc))

Out of the Classification Evaluation, the outcome of precision and recall aren't that great for True Negative values. If the prediction is to focus on only True Positive then these evaluation will be good enough. <br>

So I decided to try Accuracy and ROC/AUC. Accuracy focus on true results among the total numbers and it is good for balanced dataset and not skewed. AUC is good to measuring balanced performance of positive and negative classes. As we can see between Accuracy and AUC, Accuracy has the better result compare to AUC for predicting on customer churn. <br>

<br>

## Support Vector Machine 

#### Why not XGBoost but SVM?
At first, I was considering XGBoost but XGBoost is generally use on more large and complicated data prediction. Whereas our customer churn data is not that large and complicated, rather straightforward that consist more of categorical/classifications. <br>
<br>
In the other hand, SVM method objective is to find a hyperplane which maximized the separation between data of different classes. I see this method can be effective as our customer churn data has quite some number of features and fit our objective of to predict if customer will Exited or Not Exited (2 different classes). 


In [ ]:
svc_model = SVC(random_state=123)

svc_model.fit(X_train, y_train)
pred_svm = svc_model.predict(X_test)

In [ ]:
# Confusion Matrix 
cm = confusion_matrix(y_test, pred_svm)

sns.heatmap(cm, annot = True)

In [ ]:
print(classification_report(y_test, pred_svm))

In [ ]:
SVM_acc = accuracy_score(pred_svm, y_test) * 100
print("SVM accuracy: {}".format(SVM_acc))

In [ ]:
SVM_auc = roc_auc_score(y_test,pred_svm)*100
print("SVM AUC: {}".format(SVM_auc))

Support Vector Machines looks pretty similar but better than Logistic Regression at the accuracy score. Let's look at my 3rd choice of the algorithm - Random Forest and it's prediction.

## Random Forest

Random Forest adds additional randomness to the model, while growing the trees. Instead of searching for the most important feature while splitting a node, it searches for the best feature among a random subset of features. This results in a wide diversity that generally results in a better model.

In [ ]:
#Fitting Random Forest Classification
rf_classifier = RandomForestClassifier(random_state=123)

rf_classifier.fit(X_train, y_train)

predict_rf =rf_classifier.predict(X_test)

In [ ]:
# Confusion Matrix 
cm = confusion_matrix(y_test, predict_rf)

sns.heatmap(cm, annot = True)

In [ ]:
print(classification_report(y_test, predict_rf))

In [ ]:
#Accuracy score

rf_acc = accuracy_score(predict_rf, y_test)*100
print("Random Forest Accuracy: {}".format(rf_acc))

In [ ]:
#AUC
rf_auc = roc_auc_score(y_test, predict_rf)*100
print("Random Forest AUC: {}".format(rf_auc))

Out of the 3 algorithms that I used for the prediction models, Random Forest did the best out of them. So I want to spent a bit more focus on see how can I make improvement to the prediction. <br>

Next, I'm going to tune the model for improvement by using Cross Validation(GridSearch CV).
<br>
<br>

### Parameter Tuning

I choose GridSearch CV is because this Cross Validation model will gather all the possible combinations of parameter values are evaluated and the best combination is retained

In [ ]:
rf_classifier = RandomForestClassifier(random_state=123)

In [ ]:
param_dict_rf = {'n_estimators' : [10, 20, 40, 50, 100], 
                 'max_depth': [5, 6, 7, 8, 9, 10]}

In [ ]:
grid_rf = GridSearchCV(param_grid= param_dict_rf, estimator= rf_classifier, cv=5, verbose= 1)

In [ ]:
grid_rf.fit(X_train, y_train)

In [ ]:
grid_rf.best_params_

In [ ]:
best_rf = RandomForestClassifier(max_depth= 10, n_estimators=100, random_state=123)

model_rf = best_rf.fit(X_train, y_train)

result_rf = model_rf.predict(X_test)

In [ ]:
# Confusion Matrix 
cm = confusion_matrix(y_test, result_rf)

sns.heatmap(cm, annot = True)

In [ ]:
print(classification_report(y_test, result_rf))

In [ ]:
rf_acc = accuracy_score(result_rf, y_test)*100
print("Random Forest Accuracy: {}".format(rf_acc))

In [ ]:
rf_auc = roc_auc_score(y_test, result_rf)*100
print("Random Forest AUC: {}".format(rf_auc))

Although the improvement is not much but as we can see, the model prediction definitely improved compare to the one without the tuning.

Random Forest algorithm can help us to predict the chances of customer churn up to 86.5%. Overall it is pretty good model for the business to begin their study and prediction of customer churn.

### Feature Importance

In [ ]:
len(model_rf.feature_importances_)

In [ ]:
model_rf.feature_importances_

In [ ]:
feat_importances = pd.Series(model_rf.feature_importances_, index= X.columns)
feat_importances.nlargest(10).plot(kind='barh')

In [ ]:
feature_importances = pd.DataFrame(model_rf.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

From the Plot or Data Frame, it helps us to understand what are the main factors of customer churn based on the dataset features.
<br>
<br>
We can see and suggest that Age, Customers have 2 Products with the bank and Balance in the Bank are the top 3 most crucial factors of customer churn where their importance rates are pretty high compared to other features.

# Conclusion

- Random Forest Model is good prediction model for tackle this problem matter. But it can be improve better with some boosting methods. 
- If given more times, first thing will want to try on will be different ensemble methods and boosting algorithms to refine and improve the models. Also other different ML algorithms as those I chosen here consider the basic algorithms
- 2 of my favourite Biggest takeaways from this work
    - First ever ML model from scratch.
    - Research and reference in Kaggle how others build their model and create their notebook for better understanding.
